Lesson 15
# 課題: Web検索連動チャットボットを作成しよう

In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    # 検索ツールの定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=MODEL_NAME)
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノードの作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", chatbot)

    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付エッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition, 
    )

    # ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
    # ツールからチャットボットへの戻りエッジを作成
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    # 記憶を持つ実行可能なステートグラフの作成
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph, user_input: str):
    # 結果をストリーミングで得る
    # config引数でthread_idを指定してメモリ機能を有効化
    config = {"configurable": {"thread_id": "1"}}
    events = graph.stream({"messages": [("user", user_input)]}, config)
    for event in events:
        for value in event.values():
            if "messages" in value and value["messages"]:
                print(value["messages"][-1].content, flush=True)
    #print("\n")


# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
graph = build_graph(MODEL_NAME)

# メインループ
while True:
    user_input = input("質問:")
    if user_input.strip() == "":
        print("ありがとうございました!")
        break
    print(user_input, flush=True)
    stream_graph_updates(graph, user_input)



こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
1たす2は？
1たす2は3です。何か他に知りたいことがありますか？
台湾観光について検索結果を教えて

[{"url": "https://www.knt.co.jp/travelguide/kaigai/027/", "content": "国内\n\n海外\n\n台湾・台北観光のおすすめスポット19選！\n\n台湾・台北のツアーを日付・条件から探す\n\n台湾は日本から直行便で約4時間ほどで行ける人気の観光地。週末や連休を利用して気軽に行ける台湾は、グルメや観光、写真映えスポット、ショッピングなど幅広く楽しむことができます。国立故宮博物院や中正紀念堂など台湾の歴史あふれるエリア、若者に人気のショッピングエリアをはじめ、台北から少し足を伸ばせば、ノスタルジックな雰囲気と海を一望できる風光明媚な街並みが広がる九份があります。今回は台北を訪れたら絶対にチェックしておきたいスポットをご紹介します。\n\n目次\n\n【台北定番の観光スポット】\n\n高さ約509ｍを誇る台湾の街のシンボルタワー「台北101」\n\n世界四大博物館の1つ「国立故宮博物院」\n\n台北市で最大規模を誇る「士林観光夜市」\n\n美食がそろう夜市「饒河街観光夜市」\n\n台湾の歴史を象徴する場所「国立中正紀念堂」\n\nレトロな街並みが残る「迪化街」\n\n台湾のベニスと呼ばれる港町「淡水」\n\n商売繁盛の神様をまつる伝統的な廟「行天宮」\n\n英霊をまつる神聖な場所「忠烈祠」\n\n名店がずらりと並ぶ「永康街」"}, {"url": "https://www.hankyu-travel.com/guide/taiwan/", "content": "Image 10: 彩虹眷村彩虹眷村\n\n絶景にグルメ、ショッピング、温泉まで楽しめる台湾。台北のシンボルタワー、台北101でショッピングや眺望を楽しんだり、夜は寧夏夜市でB級グルメ三昧。足をのばして、九份の街歩きもおすすめです。そんな台湾の基本情報から観光情報まで詳しく紹介します。\n\n   観光マップ\n   おすすめ 観光スポット\n   グルメ\n   基本情報（ビザ・現地情報）\n\n台湾観光マップ\n\nMap\n\n   九份\n   台北\n   台中\n   